# Dénombrement


## Dénombrement basique


Si l'on a un groupe de 10 personnes, il y a alors $10!$ façon de les ordonner $(10\ choix) \times (9\ choix) \times \dots \times (1\ choix) = 10!$ possibilités

In [ ]:
#on veut donc pouvoir utiliser la fonction factorielle
from math import factorial

In [ ]:
#le nombre de possibilités pour ordonner notre groupe est alors :
factorial(10)

In [ ]:
#Si on voulait l'implémenter nous même on pourrait soit le faire récursivement :
def factorielle_rec(n):
  if n < 2:
    return 1
  else :
    return n * factorielle_rec(n - 1)

#soit le faire itérativement :
def factorielle_iter(n):
  produit = 1
  for i in range(2, n + 1):
    produit *= i
  return produit

In [ ]:
print(f'Factorielle récursive : {factorielle_rec(10)}\nFactorielle itérative : {factorielle_iter(10)}')

In [ ]:
%timeit factorial(100)
%timeit factorielle_rec(100)
%timeit factorielle_iter(100)

Mais c'est deux implémentations sont plus lentes en moyenne que la fonction de la librairie `math`, on continuera donc d'utiliser `factorial`.

## Dénombrement avancé

### Groupes Ordonnés



Si l'on veut maintenant dénombrer le nombre de groupes de 5 personnes **ordonnés**, on a alors $(10\ choix)×(9\ choix)×⋯×(6\ choix) = \frac{10 \times 9 \times 8 \times 7 \times 6 \times 5 \times 4 \times 3 \times 2 \times 1}{5 \times 4 \times 3 \times 2 \times 1} = \frac{10!}{5!}$ possibilités

Pour un groupe de 2 personnes **ordonné**, on aurait alors $(10\ choix)\times(9\ choix) = \frac{10 \times 9 \times 8 \times 7 \times 6 \times 5 \times 4 \times 3 \times 2 \times 1}{8 \times 7 \times 6 \times 5 \times 4 \times 3 \times 2 \times 1} = \frac{10!}{8!}$

In [ ]:
def groupe_ordonne(k, n):
  if k < 0 or k > n:
    return 0
  else :
    return factorial(n) // factorial(n - k)

In [ ]:
print(f'Groupes ordonnés de 5 personnes : {groupe_ordonne(5, 10)}\nGroupes ordonnés de 2 personnes : {groupe_ordonne(2, 10)}')

### Groupes Non Ordonnés



Il est aussi intéressant de pouvoir compter les groupes de personnes, sans prendre leur ordre en compte. Ainsi, pour un groupe de 10 personnes, on a vu tout à l'heure qu'il y avait $10!$ ordres différents. 

Ainsi, si l'on veut tous les groupes de 5 personnes **non ordonnés**, on calculera le nombre de groupes **ordonnés** de 5 personnes que l'on divisera par le nombre de manière qu'il y a de l'ordonner.


Comme vous l'avez sûrement compris, c'est la fonction `parmi`.

$C^k_n=\binom{n}{k}=\frac{n!}{k!\ \times\ (n-k)!}$

En reprenant la logique expliquée jusqu'ici, on a alors :
$\frac{nombre\ de\ possibilités\ total}{nombre\ d'ordres\ différents\ \times\ nombre\ de\ possibilités\ qu'on\ ne\ prend\ pas\ en\ compte}$

In [ ]:
def parmi_naif(k, n):
  if k < 0 or k > n:
    return 0
  else :
    return groupe_ordonne(k, n) // factorial(k)

#On a une version plus optimisée qu'on peut trouver ici : https://en.wikipedia.org/wiki/Binomial_coefficient#Binomial_coefficient_in_programming_languages
def parmi(k, n):
    if k < 0 or k > n:
        return 0
    if k == 0 or k == n:
        return 1
    k = min(k, n - k)
    c = 1
    for i in range(k):
        c = c * (n - i) // (i + 1)
    return c

In [ ]:
%timeit parmi_naif(100, 100000)
%timeit parmi(100, 100000)

In [ ]:
print(f'Groupes non ordonnés de 5 personnes : {parmi(5, 10)}\nGroupes non ordonnés de 2 personnes : {parmi(2, 10)}')

In [ ]:
#On peut aussi résoner dans l'autre sens : si on a 252 groupes différents parmi 10 personnes, alors, si on les ordonne on aura 252 groupes * le nombre de façon d'ordonner un groupe de 5 personnes soit 252 * 5!
print(f'Avec la fonction groupe_ordonné : {groupe_ordonne(5, 10)}\nAvec la fonction parmi : {parmi(5, 10) * factorial(5)}\nSans fonction : {factorial(10) // factorial(5)}')

## Dénombrement Avec Remise

### Groupes Ordonnés

On retouve ici les mêmes équations que tout à l'heure. Alors que pour le nombre de groupes ordonnés, la réponse est simple, on remplace la factorielle par une puissance, la fonction `parmi_avec_remise` est beaucoup plus compliquée...

In [ ]:
#Nombre de groupe de k objets avec n objets au total
def groupe_ordonne_avec_remise(k, n):
  return n**k

In [ ]:
#On voit ici qu'on a le même résultat que pour groupe_ordonne(2, 10) auquel on rajoute les 10 groupes : (1, 1) (2, 2), ..., (10, 10) soit 90 + 10
groupe_ordonne_avec_remise(2, 10)

### Groupes Non Ordonnés

On veut ici dénombrer le nombre de résultats possible, sans tenir compte de l'ordre

In [ ]:
#On peut commencer par faire une version qui ne calcule pas, mais test directement et nous renvoie le nombre de groupes différents
def parmi_avec_remise_simulation(k, n):
  if k == 0:
    return 1
  if k < 0:
    return 0
  tirages_ordonnés = [[i] for i in range(n)]
  for k in range(k - 1):
    new_tirages = []
    for tirage in tirages_ordonnés:
      # pour chaque tirage déjà existant
      for i in range(n):
        #on rajoute toutes les possibilités pour le k-ième tirage
        new_tirages.append(tirage + [i])
    
    tirages_ordonnés = new_tirages
  
  #une fois qu'on a tous nos tirages possibles, il ne reste qu'à les regrouper pour n'avoir que des tirages différents
  #pour cela, on va tous les trier afin que 2 tirages ayant les mêmes évènements (même dans un ordre différent), soient vus comme identiques

  [tab.sort() for tab in tirages_ordonnés]
  #on utilise ici deux compréhensions : https://docs.python.org/fr/3/tutorial/datastructures.html#list-comprehensions
  #                                     https://docs.python.org/fr/3/tutorial/datastructures.html#sets

  #la première ne retourne rien, elle permet juste de trier les listes de tirage
  #ensuite, on a donc une grande liste contenant pleins de listes de tirages triés
  #on transforme chaque liste de tirage en tuple [tirage1, tirage2, ... tiragek] => (tirage1, tirage2, ... tiragek)
  #puis on ajoute ces tuples dans un ensemble qui supprime tous les doublons, ainsi, il ne reste plus que les tirages différents
  tirages_non_ordonnés = {tuple(tab) for tab in tirages_ordonnés}

  #on renvoie alors le nombre d'élément dans l'ensemble, soit le nombre de tirages différents non ordonnés
  return len(tirages_non_ordonnés)

#bien évidemment, le nombre de calcul explose pour un k et un n trop grand, on gère en effet n**k éléments, on a donc une complexité exponentielle

#vous pouvez par exemple essayer parmi_avec_remise_simulation(7, 10) qui va presque faire exploser le noyau python pour un résultat ... assez petit en fin de compte



Si on veut maintenant faire une version plus calculatoire, on peut choisir de se baser sur le nombre d'évènements identiques dans chaque tirage.

Par exemple, pour un tirage avec remise de 4 éléments parmi 10 il faut prendre en compte : 
* 4 objets différents : $10 \times 9 \times 8 \times 7 = C^4_{10}$ tirages.
* 3 objets différents : $\frac{10 \times 9 \times 8}{2!} = C^3_{10} \times 3$ tirages. Il ne faut pas oublier que les 2 premiers objets différents peuvent potententiellement être ordonnés de n'importe quelle manière sans changer le résultat final, la où si on échange l'objet en double avec un objet seul, on change de tirage non ordonné. C'est pour cela qu'on divise  par $2!$.
* 2 objets différents : 
    * Deux groupes de 2 objets : $\frac{10 \times 9}{2!}=C^2_{10}$.
    * Un groupe de 3 objets    : $10 \times 9 = C^2_{10} \times 2$.
* 4 objets identiques : $10$

In [ ]:
#C'est la qu'on voit que la simulation est vraiment pas mal pour vérifier nos résultats.
#Ce qui est vraiment bien avec Python, c'est qu'on peut vraiment simuler tout et n'importe quoi assez facilement. 

#Ainsi, si l'on vérifie ce que j'avance au-dessus on a alors:
print(f'Par le calcul : {10 + parmi(2, 10) * 2 + parmi(2, 10) + parmi(3, 10) * 3 + parmi(4, 10)}\nPar simulation : {parmi_avec_remise_simulation(4,10)}')

Notre problème revient alors à regrouper nos k éléments en autant de manière que l'on peut, puis de calculer le nombre de combinaisons que chaque regroupement créer (chaque regroupement contenant des éléments identiques). Pour 5 éléments parmi 10, on a alors :
* $(1, 1, 1, 1, 1)=> C^5_{10}$
* $(1, 1, 1, 2)\ \ \ \ => C^4_{10} \times 4$
* $(1, 2, 2)^1\ \ \ \ \ \ => C^3_{10} \times \frac{3!}{2!}$
* $(1, 1, 3)\ \ \ \ \ \ \ \ => C^3_{10} \times 3$
* $(2, 3)\ \ \ \ \ \ \ \ \ \ \ \ => C^2_{10} \times 2$
* $(1, 4)\ \ \ \ \ \ \ \ \ \ \ \ => C^2_{10} \times 2$
* $(5)\ \ \ \ \ \ \ \ \ \ \ \ \ \ \ \ => C^1_{10}$

$^1$ : pour $(1, 2, 2)$, on a $3!$ manières des les ordonner mais sachant qu'on peut échanger l'ordre des deux groupe de 2, il faut alors diviser par $2!$.

In [ ]:
#Ainsi, si l'on vérifie ce que j'avance au-dessus on a alors:
print(f'Par le calcul : {parmi(5, 10) + parmi(4, 10) * 4 + parmi(3, 10) * 3 + parmi(3, 10) * 3 + parmi(2, 10) * 2 + parmi(2, 10) * 2 + parmi(1, 10)}\nPar simulation : {parmi_avec_remise_simulation(5,10)}')

In [ ]:
#On veut donc créer une fonction qui crée tous les groupements de k éléments possibles
def groupements(k, n, taille_max=None):
  if k == 0 :
    return [[]]

  #On peut rajouter une optimisation ici, en effet, on ne peut pas avoir plus de groupes qu'il n'y a de valeurs
  #ainsi, si on a atteint le nombre de groupe maximum on peut arrêter la récursion. Ca allègera le programme.
  #if nombre_de_groupes == n:
  #  return []

  if k == 1:
    return [[1]]

  groupes = []
  if taille_max == None:
    taille_max = k
  for i in range(1, min(taille_max + 1, k + 1)):
    #on peut rajouter une optimisation ici, en s'assurant qu'il est toujours possible de faire au plus n groupes
    #si toutes les prochaines valeurs des groupes seront au maximum i
    if (k - i) - (n - 1) * i <= 0:
      [groupes.append(groupe + [i]) for groupe in groupements(k - i, n - 1, i)]
  
  return groupes

In [ ]:
#On retrouve ce qu'on avait anticipé ci-dessus
groupements(5, 5)

In [ ]:
#Il faut maintenant calculer le nombre de combinaisons différentes peut former un groupe
def parmi_groupe(groupe, n):
  base = groupe_ordonne(len(groupe), n)
  #il ne reste plus qu'à enlever l'ordre
  #pour chaque taille de groupe, on compte combien on en a
  #et on par du principe qu'on peut interchanger les groupes de même taille

  #On peut faire ça simplement sans trop optimiser
  #for taille in set(groupe):
  #  base //= factorial(groupe.count(taille))

  #Ou en optimisant un peu plus:
  d = {}
  for taille in groupe:
    if taille in d:
      d[taille] += 1
    else:
      d[taille] = 1
  for count in d.values():
    base //= factorial(count)

  return base


In [ ]:
#Il ne nous reste plus qu'à utiliser tout ce qu'on a implémenté

def parmi_avec_remise_groupements(k, n):
  return sum([parmi_groupe(groupe, n) for groupe in groupements(k, n)])

In [ ]:
%timeit parmi_avec_remise_simulation(5,10)
%timeit parmi_avec_remise_groupements(5, 10)

print(f'Par simulation : {parmi_avec_remise_simulation(5,10)}\nPar le calcul : {parmi_avec_remise(5, 10)}\n')

Il est maintenant temps d'implémenter une formule plus direct. Il est assez simple de la prouver par récurrence.

Soit les $x_1, x_2, ... , x_n$ éléments possibles.

Soit $(k_1, k_2, ..., k_k)$ un tirage de k éléments triés ($x_1 sera toujours avant x_2$) pouvant se répéter parmi les $x_1, x_2, ... , x_n$.

Soit $\Gamma^{k}_{n}$ la fonction renvoyant le nombre de combinaisons possibles pour un tirage non ordonné de k éléments parmi n avec répétition.

On peut alors avancer deux hypothèses :
* Soit k_1 est égale à x_1, on a alors $\Gamma^{k - 1}_{n}$ combinaisons restantes possibles.
* Soit k_1 est différent de x_1, ainsi, notre tirage ne contient pas x_1, on a alors $\Gamma^{k}_{n - 1}$ combinaisons restantes possibles.

Ainsi, par récurrence on a :
    
$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \Gamma^{k}_{n} = \Gamma^{k - 1}_{n} + \Gamma^{k}_{n - 1}$ avec $\Gamma^{k}_{1} = 1$ et $\Gamma^{0}_{n} = 1$.


On peut alors retrouver :

$\ \ \ \ \ \ \ \ \ \ \ \ \ \ \Gamma^{k}_{n} = \frac{(n + k - 1)!}{k! (n - 1)!} = C^{k}_{n + k - 1}$

In [ ]:
def parmi_avec_remise(k, n):
  return parmi(k, k + n - 1)

In [ ]:
%timeit parmi_avec_remise_groupements(50, 100)
%timeit parmi_avec_remise(50, 100)

print(f'Par groupement : {parmi_avec_remise_groupements(50, 100)}\nPar la formule : {parmi_avec_remise(50, 100)}\n')